In [1]:
import itertools
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

### DataSource

In [2]:
class DataSource:
    
    def __init__(self, filename):
        self.df = pd.read_csv(filename)
        
    def data_load_split(self, target=None, ignore=None):
        self.target = target
        self.ignore = ignore
        self.inputs = sorted(set(self.df.columns) - set(self.target) - set(self.ignore))
        
        self.X = self.df[self.inputs]
        self.y = self.df[self.target]
        
        return self.X, self.y
    
    def define_problem(self):
        if self.y.dtypes[0] in ['int64', 'float64'] and self.y.nunique()[0] == 2:
            self.problem = "Binary"
        elif self.y.dtypes[0] in ['object', 'bool']:
            self.problem = "Classification"
        else:
            self.problem = "Regression"
    
        return self.problem
    
    def data_preprocess(self, X, y, problem="Regression"):

        # Data type detection
        numerical_ix = self.X.select_dtypes(include=['int64', 'float64']).columns
        categorical_ix = self.X.select_dtypes(include=['object', 'bool']).columns

        # Data transform
        num_transform = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ])
        cat_transform = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value="Missing")),
            ('oh_encoder', OneHotEncoder(sparse=False))
        ])

        transform_x = ColumnTransformer(transformers=[
            ('num', num_transform, numerical_ix),
            ('cat', cat_transform, categorical_ix)
        ])
        
        if problem == "Regression" or "Binary":
            transform_y = ColumnTransformer(transformers=[
                ('num', Normalizer(), y.columns)
            ])
        else:
            transform_y = ColumnTransformer(transformers=[
                ('cat', cat_transform, y.columns)
            ])
            
        self.trans_X = transform_x.fit_transform(self.X)
        self.trans_y = transform_y.fit_transform(self.y)

        return self.trans_X, self.trans_y
    
    def train_val_split(self, X, y, ratio=0.2, random_state=42):
        return train_test_split(X, y, test_size=ratio, random_state=random_state)

In [3]:
filename="./data/Churn_Modelling.csv"
ds = DataSource(filename)

In [5]:
ds.df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [6]:
X, y = ds.data_load_split(target=['Exited'], 
                          ignore=["RowNumber", "CustomerId", "Surname"])

problem = ds.define_problem()
trans_X, trans_y = ds.data_preprocess(X, y, problem=problem)
X_train, X_val, y_train, y_val = ds.train_val_split(trans_X, trans_y, ratio=0.2, random_state=42)

### MLP

In [7]:
class MLP:
    
    def __init__(self, problem="Regression"):
        self.problem = problem
        tf.random.set_seed(42)
    def build_structure(self, max_hidden_layers=1, units=[16]):
        self.structures = []
        self.structures_info = []
        self.max_hidden_layers = max_hidden_layers
        self.units = units
    
        grid = [np.arange(self.max_hidden_layers)+1, self.units]
        for param_tuple in itertools.product(*grid):
            structure_param = {'hidden_layers': param_tuple[0],
                                'units': param_tuple[1]}

            # input layer
            model = keras.Sequential()
            model.add(keras.layers.Dense(16, input_shape=(trans_X.shape[1],)))

            # hidden layer block
            for _ in range(structure_param['hidden_layers']):
                model.add(keras.layers.Dense(structure_param['units'], activation='relu'))

            # output layer
            if problem == 'Regression':
                model.add(keras.layers.Dense(1))
            elif problem == 'Binary':
                model.add(keras.layers.Dense(1, activation='sigmoid'))
            else:
                model.add(keras.layers.Dense(trans_y.shape[1], activation='softmax'))
            self.structures.append(model)
            self.structures_info.append(structure_param)

        return self.structures, self.structures_info
    
    def create_optimizer(self, optimizers=['adam'], lrs=[0.01], use_all=False):
        self.created_optimizers = []
        self.optimizers_info = []
        self.optimizers = optimizers
        self.lrs = lrs

        self.optimizer_classes = {'adadelta': keras.optimizers.Adadelta, 'sgd': keras.optimizers.SGD,
                                  'adam': keras.optimizers.Adam, 'adagrad': keras.optimizers.Adagrad,
                                  'adamax': keras.optimizers.Adamax, 'rmsprop': keras.optimizers.RMSprop}

        if use_all:
            self.lrs = [0.001, 0.01, 0.02, 0.1]
            opt_grid = [self.optimizer_classes.keys(), self.lrs]    
        else:
            opt_grid = [self.optimizers, self.lrs]

        for opt_tuple in itertools.product(*opt_grid):
            opt_param = {
                'optimizer_name': opt_tuple[0],
                'lr': opt_tuple[1]
            }

            opt_class = self.optimizer_classes.get(opt_param['optimizer_name'])
            self.created_optimizers.append(opt_class(opt_param['lr']))
            self.optimizers_info.append(opt_param)

        return self.created_optimizers, self.optimizers_info
    
    def _compile_model(self):
        if self.problem == "Regression":
            self.loss = keras.losses.MSE
            self.metrics = ['MSE', 'MAE']
        elif self.problem == "Binary":
            self.loss = keras.losses.binary_crossentropy
            self.metrics = ['accuracy']
        else:
            self.loss = keras.losses.categorical_crossentropy
            self.metrics = ['accuracy']

        self.compiled_models = []
        self.compiled_models_info = []

        compile_grid = [zip(self.structures, self.structures_info), zip(self.created_optimizers ,self.optimizers_info)]
        for compile_tuple in itertools.product(*compile_grid):
            compile_param = {'model': compile_tuple[0][0],
                             'optimizer': compile_tuple[1][0]}
            model_info = {'structure_info': compile_tuple[0][1],
                           'optimizer_info': compile_tuple[1][1]}

            model = compile_param['model']
            model.compile(optimizer=compile_param['optimizer'],
                          loss=self.loss,
                          metrics=self.metrics)

            self.compiled_models.append(model)
            self.compiled_models_info.append(model_info)

        return self.compiled_models, self.compiled_models_info
    
    def train_models(self, models, X_train, y_train, X_val=None, y_val=None,
                     batch_size=None, epochs=1, verbose=0, callbacks=None,
                     shuffle=True, steps_per_epoch=None):
        self.models = models
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val

        self.trained_models = []
        self.histories = []
        self.val_losses = []
        for model in models:
            history = model.fit(x=self.X_train, y=self.y_train,
                                batch_size=batch_size, epochs=epochs,
                                verbose=verbose, callbacks=callbacks,
                                validation_data=(self.X_val, self.y_val), shuffle=shuffle)
            val_loss = model.evaluate(self.X_val, self.y_val, verbose=0)
            self.trained_models.append(model)
            self.histories.append(history)
            self.val_losses.append(val_loss[0])
            print("{} model is trained. best validation loss is : {}".format(
                model.name, val_loss))
            
        return self.trained_models

In [9]:
mlp = MLP(problem=problem)
structures, structures_info = mlp.build_structure(max_hidden_layers=3, units=[16, 32, 64, 128, 256])
created_optimizers, optimizers_info = mlp.create_optimizer(optimizers=['adam'], lrs=[0.1], use_all=True)
compiled_models, compiled_models_info = mlp._compile_model()

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
trained_models = mlp.train_models(compiled_models,
                                  X_train, y_train, X_val, y_val,
                                  batch_size=64, epochs=20,
                                  callbacks=[callback])

sequential_15 model is trained. best validation loss is : [0.391639696598053, 0.826]
sequential_15 model is trained. best validation loss is : [0.3879754247665405, 0.8385]
sequential_15 model is trained. best validation loss is : [0.37795215606689453, 0.8495]
sequential_15 model is trained. best validation loss is : [0.3957252221107483, 0.831]
sequential_15 model is trained. best validation loss is : [0.38717878007888795, 0.84]
sequential_15 model is trained. best validation loss is : [0.3905567789077759, 0.844]
sequential_15 model is trained. best validation loss is : [0.38060651350021363, 0.851]
sequential_15 model is trained. best validation loss is : [0.37306646108627317, 0.856]
sequential_15 model is trained. best validation loss is : [0.3667350180149078, 0.856]
sequential_15 model is trained. best validation loss is : [0.392258406162262, 0.84]
sequential_15 model is trained. best validation loss is : [0.37541975450515747, 0.8585]
sequential_15 model is trained. best validation lo

sequential_18 model is trained. best validation loss is : [0.466597029209137, 0.823]
sequential_19 model is trained. best validation loss is : [0.4035784664154053, 0.825]
sequential_19 model is trained. best validation loss is : [0.42535706996917727, 0.8215]
sequential_19 model is trained. best validation loss is : [0.4496127815246582, 0.8195]
sequential_19 model is trained. best validation loss is : [0.43761937236785886, 0.8135]
sequential_19 model is trained. best validation loss is : [0.40323046493530273, 0.8195]
sequential_19 model is trained. best validation loss is : [0.4694702434539795, 0.821]
sequential_19 model is trained. best validation loss is : [0.46648512601852415, 0.8215]
sequential_19 model is trained. best validation loss is : [0.4681059727668762, 0.821]
sequential_19 model is trained. best validation loss is : [0.4660923295021057, 0.821]
sequential_19 model is trained. best validation loss is : [0.46539560890197756, 0.8225]
sequential_19 model is trained. best validat

sequential_22 model is trained. best validation loss is : [0.4062987167835236, 0.8035]
sequential_22 model is trained. best validation loss is : [0.40629677605628967, 0.8035]
sequential_23 model is trained. best validation loss is : [0.43648675036430357, 0.8035]
sequential_23 model is trained. best validation loss is : [0.44006101942062376, 0.8035]
sequential_23 model is trained. best validation loss is : [0.4357242856025696, 0.8045]
sequential_23 model is trained. best validation loss is : [0.43853677797317503, 0.803]
sequential_23 model is trained. best validation loss is : [0.408238774061203, 0.818]
sequential_23 model is trained. best validation loss is : [0.4253806643486023, 0.8195]
sequential_23 model is trained. best validation loss is : [0.43112416911125184, 0.814]
sequential_23 model is trained. best validation loss is : [0.4132912797927856, 0.8155]
sequential_23 model is trained. best validation loss is : [0.410419517993927, 0.816]
sequential_23 model is trained. best validat

sequential_26 model is trained. best validation loss is : [0.4944820680618286, 0.8035]
sequential_26 model is trained. best validation loss is : [0.49448217725753785, 0.8035]
sequential_26 model is trained. best validation loss is : [0.49448227071762085, 0.8035]
sequential_27 model is trained. best validation loss is : [0.48843998861312865, 0.808]
sequential_27 model is trained. best validation loss is : [0.4955122718811035, 0.8035]
sequential_27 model is trained. best validation loss is : [0.4955122718811035, 0.8035]
sequential_27 model is trained. best validation loss is : [0.4955122718811035, 0.8035]
sequential_27 model is trained. best validation loss is : [0.4955122718811035, 0.8035]
sequential_27 model is trained. best validation loss is : [0.4955122718811035, 0.8035]
sequential_27 model is trained. best validation loss is : [0.4955122718811035, 0.8035]
sequential_27 model is trained. best validation loss is : [0.4955122718811035, 0.8035]
sequential_27 model is trained. best vali

In [11]:
idx = np.argmin(mlp.val_losses)
best_model = compiled_models[idx]
best_model_info = compiled_models_info[idx]

In [12]:
best_model

In [13]:
best_model_info

{'structure_info': {'hidden_layers': 1, 'units': 16},
 'optimizer_info': {'optimizer_name': 'adam', 'lr': 0.001}}

In [14]:
best_model.evaluate(X_val, y_val)

63/63 [==============================] - 0s 988us/step - loss: 0.4011 - accuracy: 0.8630


[0.4011317610740662, 0.863]

In [15]:
mlp.val_losses[idx]

0.3667350180149078